In [1]:
from WattPredictor.utils.helpers import *
from WattPredictor.constants import *
from WattPredictor.utils.exception import *
from WattPredictor.utils.logging import logger
from pathlib import Path
from dataclasses import dataclass


In [2]:
import os
import sys
os.chdir('../')
sys.path.append(os.path.join(os.getcwd(), "src"))

In [3]:
import sys
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
from WattPredictor.utils.logging import logger
from WattPredictor.utils.exception import CustomException
from WattPredictor.utils.feature import feature_store_instance


class Predictor:
    def __init__(self, model_name: str, model_version: int, feature_view_name: str, feature_view_version: int, n_features: int):
        self.feature_store = feature_store_instance()
        self.model_name = model_name
        self.model_version = model_version
        self.feature_view_name = feature_view_name
        self.feature_view_version = feature_view_version
        self.n_features = n_features
        self.model = self.feature_store.load_model(
            model_name=self.model_name,
            model_version=self.model_version,
            model_filename='model.joblib'
        )

    def _load_batch_features(self, current_date: datetime) -> pd.DataFrame:
        try:
            feature_view = self.feature_store.feature_store.get_feature_view(
                name=self.feature_view_name,
                version=self.feature_view_version
            )

            fetch_data_to = current_date - timedelta(hours=1)
            fetch_data_from = current_date - timedelta(days=28)

            ts_data = feature_view.get_batch_data(
                start_time=fetch_data_from,
                end_time=fetch_data_to
            )

            ts_data = ts_data.groupby('sub_region_code').tail(self.n_features)
            ts_data.sort_values(by=['sub_region_code', 'date'], inplace=True)

            location_ids = ts_data['sub_region_code'].unique()
            x = np.ndarray((len(location_ids), self.n_features), dtype=np.float32)
            temperature_values = []

            for i, loc in enumerate(location_ids):
                sub_data = ts_data[ts_data['sub_region_code'] == loc]
                demand_series = sub_data['demand'].values[-self.n_features:]

                if len(demand_series) < self.n_features:
                    logger.warning(f"Padded {loc}: {len(demand_series)} available, padding to {self.n_features}.")
                    demand_series = np.pad(demand_series, (self.n_features - len(demand_series), 0), 'constant', constant_values=0)

                x[i, :] = demand_series
                temperature_values.append(sub_data['temperature_2m'].iloc[-1])

            features = pd.DataFrame(
                x, columns=[f'demand_previous_{i+1}_hour' for i in reversed(range(self.n_features))]
            )
            features['temperature_2m'] = temperature_values
            features['date'] = current_date
            features['sub_region_code'] = location_ids

            logger.info(f"Features generated for {len(location_ids)} regions.")
            return features

        except Exception as e:
            logger.error("Failed to load batch features.")
            raise CustomException(e, sys)

    def predict(self, current_date: datetime, save_to_store: bool = False) -> pd.DataFrame:
        try:
            features = self._load_batch_features(current_date)
            feature_input = features.drop(columns=['date'], errors='ignore')
            predictions = self.model.predict(feature_input)

            results = pd.DataFrame({
                'sub_region_code': features['sub_region_code'],
                'predicted_demand': predictions.round(0),
                'date': features['date']
            })

            logger.info("Predictions generated successfully.")

            if save_to_store:
                self.save_predictions_to_store(results)

            return results

        except Exception as e:
            logger.error("Prediction process failed.")
            raise CustomException(e, sys)

    def save_predictions_to_store(self, predictions: pd.DataFrame):
        try:
            if predictions.empty:
                logger.warning("No predictions to save.")
                return

            self.feature_store.create_feature_group(
                name='elec_wx_predictions',
                df=predictions,
                primary_key=["sub_region_code"],
                event_time="date",
                description="Predicted electricity demand",
                online_enabled=True
            )

            logger.info("Predictions saved to feature store.")

        except Exception as e:
            logger.error("Failed to save predictions to feature store.")
            raise CustomException(e, sys)

In [4]:
if __name__ == "__main__":
    predictor = Predictor(
        model_name='wattpredictor_lightgbm',
        model_version=1,
        feature_view_name='elec_wx_features_view',
        feature_view_version=1,
        n_features=672
    )

    current_date = datetime(2025, 5, 20)

    predictions = predictor.predict(current_date=current_date, save_to_store=True)
    print(predictions)

[2025-07-17 18:28:33,007: INFO: helpers: yaml file: config_file\config.yaml loaded successfully]
[2025-07-17 18:28:33,013: INFO: helpers: yaml file: config_file\params.yaml loaded successfully]
[2025-07-17 18:28:33,016: INFO: helpers: yaml file: config_file\schema.yaml loaded successfully]
[2025-07-17 18:28:33,019: INFO: external: Initializing external client]
[2025-07-17 18:28:33,020: INFO: external: Base URL: https://c.app.hopsworks.ai:443]
[2025-07-17 18:28:38,097: INFO: python: Python Engine initialized.]

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1240214
[2025-07-17 18:28:40,784: INFO: feature_store: Connected to Hopsworks Feature Store: WattPredictor]


Downloading: 0.000%|          | 0/2154054 elapsed<00:00 remaining<?

[2025-07-17 18:28:45,905: INFO: feature_store: Model 'wattpredictor_lightgbm' v1 loaded successfully.]
Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (2.14s) 
[2025-07-17 18:28:53,225: WARNING: 1201174055: Padded 0: 671 available, padding to 672.]
[2025-07-17 18:28:53,226: WARNING: 1201174055: Padded 1: 671 available, padding to 672.]
[2025-07-17 18:28:53,229: WARNING: 1201174055: Padded 2: 671 available, padding to 672.]
[2025-07-17 18:28:53,231: WARNING: 1201174055: Padded 3: 671 available, padding to 672.]
[2025-07-17 18:28:53,233: WARNING: 1201174055: Padded 4: 671 available, padding to 672.]
[2025-07-17 18:28:53,235: WARNING: 1201174055: Padded 5: 671 available, padding to 672.]
[2025-07-17 18:28:53,237: WARNING: 1201174055: Padded 6: 671 available, padding to 672.]
[2025-07-17 18:28:53,239: WARNING: 1201174055: Padded 7: 671 available, padding to 672.]
[2025-07-17 18:28:53,241: WARNING: 1201174055: Padded 8: 671 available, padding to 672.]
[2025-07-1

Uploading Dataframe: 100.00% |██████████| Rows 11/11 | Elapsed Time: 00:01 | Remaining Time: 00:00


Launching job: elec_wx_predictions_2_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai:443/p/1240214/jobs/named/elec_wx_predictions_2_offline_fg_materialization/executions
[2025-07-17 18:29:14,832: INFO: feature_store: Feature Group 'elec_wx_predictions' v2 created successfully.]
[2025-07-17 18:29:14,833: INFO: 1201174055: Predictions saved to feature store.]
    sub_region_code  predicted_demand       date
0                 0            1616.0 2025-05-20
1                 1             744.0 2025-05-20
2                 2            1297.0 2025-05-20
3                 3             548.0 2025-05-20
4                 4             651.0 2025-05-20
5                 5            1248.0 2025-05-20
6                 6             985.0 2025-05-20
7                 7             290.0 2025-05-20
8                 8             641.0 2025-05-20
9                 9            5648.0 2025-05-20
10               10            2222.0